# Badanie intensywności procesów punktowych

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def point_count_on_subregions(points, bins, x_lim, y_lim):
    xpoints=np.linspace(x_lim[0], x_lim[1], bins[0]+1)
    ypoints=np.linspace(y_lim[0], y_lim[1], bins[1]+1)
    
    H, xedges, yedges = np.histogram2d(points["X"], points["Y"], bins=(xpoints, ypoints))
    H = H.T
    return [xpoints,ypoints,H]

points_1=pd.read_csv('points_1.csv')
points_2=pd.read_csv('points_2.csv')
points_3=pd.read_csv('points_3.csv')

Test chi-kwadrat Pearsona

In [3]:
def distribution_table(bin_counts):
    """
    Parameters
    -------
    bin_counts: array
        Macierz 2D z liczbą punków przypisanych do każdego z podobszarów.

    Returns
    -------
    table: DataFrame
        Tablica zawierająca 2 kolumny:
        - "K", która zawiera wszystkie wartości całkowite z zakresu od minimalnej do maksymalnej liczby zliczeń w obrębie podobszarów,
        - "N(K)", która zawiera liczby podobszarów, którym zostały przypisane poszczególne liczby punktów.
    """    
    min_ob=bin_counts.min()
    max_ob=bin_counts.max()
    K=np.arange(min_ob,max_ob+1)
    n=[]
    for i in K:
        n.append(np.sum(bin_counts==i))
    return pd.DataFrame({"K":K.flatten(), "N(K)":n})

def poisson_distribution_table(k, mu):
    """
    Parameters
    -------
    k: array
        Macierz 1D z wariantami badanej cechy, dla którym ma zostać wyliczone prawdopodobieństwo.
    mu: int
        Wartość oczekiwana rozkładu Poissona.

    Returns
    -------
    table: DataFrame
        Tablica zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy,
        - "P(K)", która zawiera wartości prawdopodobieństw rozkładu Poissona wyliczone dla wartości oczekiwanej mu
        oraz poszczególnych wariantów badanej cechy znormalizowane do sumy wartości równej 1.
    """  
    p=sp.stats.poisson.pmf(k, mu)
    p=p/np.sum(p)
    return pd.DataFrame({"K":k, "P(K)":p})

def pearsons_chi2_test(tested_distribution, theoretical_distribution, alpha, ddof):
    """
    Parameters
    -------
    tested_distribution: DataFrame
        Tablica opisująca testowany rozkład i zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy, wartości muszą być identycznej jak kolumna "K" zmiennej lokalnej theoretical_distribution,
        - "N(K)", która zawiera liczebności poszczególnych wariantów badanej cechy.

    theoretical_distribution: DataFrame
        Tablica opisująca rozkład teoretyczny i zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy, wartości muszą być identycznej jak kolumna "K" zmiennej lokalnej tested_distribution,
        - "P(K)", która zawiera prawdopodobieństwa poszczególnych wariantów badanej cechy. Wartości z tej kolumny muszą sumować się do 1.
    
    alpha: float
        Wartość z zakresu [0,1] określająca poziom istotności.
    
    ddof: int
        Liczba nieujemna określająca liczbę nieznanych parametrów rozkładu.
    """
    chi_kwadrat=0
    n=0
    s=np.size(tested_distribution["K"])
    n=sum(tested_distribution["N(K)"])
    chi_kwadrat=np.sum((tested_distribution["N(K)"]-n*theoretical_distribution["P(K)"])**2/n/theoretical_distribution["P(K)"])
    chi_kwadrat_alfa=sp.stats.chi2.ppf(1-alpha,s-1)
    
    print("Test chi-kwadrat Pearsona")
    print("H0: Testowana zmienna ma przyjęty rozkład teoretyczny")
    print("H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego")
    print("chi2 = {} chi2_alfa = {}".format(chi_kwadrat, chi_kwadrat_alfa))
    if(chi_kwadrat<chi_kwadrat_alfa):
        print("chi2 < chi2_alfa")
        print("Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = {}".format(alpha))
    else:
        print("chi2 >= chi2_alfa")
        print("Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = {}".format(alpha))

count_1=point_count_on_subregions(points_1, [40,20],[0,20],[0,10])[2].T
dt1=distribution_table(count_1)
dpt1=poisson_distribution_table(dt1["K"],20*(20*10)/(40*20))
pearsons_chi2_test(dt1, dpt1, 0.05, 0)

count_2=point_count_on_subregions(points_2, [40,20],[0,20],[0,10])[2].T
dt2=distribution_table(count_2)
dpt2=poisson_distribution_table(dt2["K"],20*(20*10)/(40*20))
pearsons_chi2_test(dt2, dpt2, 0.05, 0)

Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = 17.775385349094424 chi2_alfa = 22.362032494826934
chi2 < chi2_alfa
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05
Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = 86.27901230036937 chi2_alfa = 22.362032494826934
chi2 >= chi2_alfa
Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05


Test Kołmogorowa - Smirnowa

In [4]:
def kolmogorow_smirnow_test(tested_points, theoretical_points, alpha, ddof):
    """
    Parameters
    -------
    tested_points: DataFrame
        Tablica zawierająca kolumnę ze współrzędnymi punktów testowanego rozkładu opisaną jako "X" lub "Y".

    theoretical_points: DataFrame
        Tablica zawierająca kolumnę ze współrzędnymi punktów toeretycznego rozkładu opisaną jako "X" lub "Y".
    
    alpha: float
        Wartość z zakresu [0,1] określająca poziom istotności.
    
    ddof: int
        Liczba nieujemna określająca liczbę nieznanych parametrów rozkładu.
    """
    l_a=sp.stats.kstwobign.ppf(0.95)
    D, pvalue = sp.stats.kstest(tested_points.squeeze(),theoretical_points.squeeze())
    l = D*tested_points.size**0.5
    print("Test Kołmogorowa-Smirnowa dla współrzędnej {}".format(tested_points.columns[0]))
    print("H0: Testowana zmienna ma przyjęty rozkład teoretyczny")
    print("H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego")
    print("lambda = {} lambda_alfa = {}".format(round(l,3), round(l_a,3)))
    if(l<l_a):
        print("lambda < D_alfa")
        print("Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = {}".format(alpha))
    else:
        print("lambda >= D_alfa")
        print("Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = {}".format(alpha))
    
theoretical_points=np.linspace(0,20,np.size(points_3["X"]))
kolmogorow_smirnow_test(points_3[["X"]], theoretical_points, 0.05, 0)
theoretical_points=np.linspace(0,10,np.size(points_3["Y"]))
kolmogorow_smirnow_test(points_3[["Y"]], theoretical_points, 0.05, 0)

Test Kołmogorowa-Smirnowa dla współrzędnej X
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = 2.156 lambda_alfa = 1.358
lambda >= D_alfa
Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05
Test Kołmogorowa-Smirnowa dla współrzędnej Y
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = 0.571 lambda_alfa = 1.358
lambda < D_alfa
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05
